<a href="https://colab.research.google.com/github/ntrajic/generative-nlp-with-variational-autoencoders-3834359/blob/main/Chapter1/Exercises/Chapter1Challenge_SiameseNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge: Create a Siamese Network with Custom Layers, Custom Losses, and Custom Outputs

Creating a Siamese Network using Keras to determine if two MNIST images are of the same digit involves several steps. A Siamese Network typically consists of two identical subnetworks with shared weights. The output is a measure of similarity between the inputs. For this task, we'll also add a custom layer and a custom loss function suitable for comparing similarity.

Here are the steps we'll follow:

1. Load the MNIST Dataset: We'll use the MNIST dataset available in Keras.

1. Define the Custom Layer: This could be a simple layer for demonstration purposes.

1. Define the Siamese Network Architecture: The architecture will consist of two identical subnetworks.

1. Implement a Custom Loss Function: Suitable for a Siamese network, typically a contrastive loss function.

1. Prepare the Data: Format the MNIST data for the Siamese network training.

1. Compile and Train the Model: Using the custom loss function.


Here is a nice image to represent a Siamese Network:


![](https://pyimagesearch.com/wp-content/uploads/2020/11/keras_siamese_networks_header.png)

The difference in our case is that we are not going to use a ConvNet but a normal Fully Connected network with a custom layer, and that at the end we are not going to apply the sigmoid loss so our output will be the euclidean distance between the images, a low value represents images being equal and a high value being different

In [2]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda, Layer
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# 1. Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.



In [3]:
# 2. Custom Layer (for demonstration)
class CustomLayer(Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w)

# 3. Base network to be shared (Siamese)
def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = CustomLayer(64)(x)  # Custom layer
    return Model(input, x)

# The shape of individual input images
input_shape = x_train.shape[1:]

# Create the base network
base_network = create_base_network(input_shape)

# Create the left input and point to the base network
input_a = Input(shape=input_shape)
processed_a = base_network(input_a)

# Create the right input and point to the base network
input_b = Input(shape=input_shape)
processed_b = base_network(input_b)

# Custom Layer or Function to compute the distance
def euclidean_distance(vectors):
    a, b = vectors
    sum_square = tf.reduce_sum(tf.square(a - b), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon()))

distance = Lambda(euclidean_distance)([processed_a, processed_b])

# Create the Siamese Network model
model = Model([input_a, input_b], distance)

In [4]:
# 4. Custom Contrastive Loss Function
def contrastive_loss(y_true, y_pred):
    margin = 1
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

In [5]:
# 5. Function to create pairs
def create_pairs(x, digit_indices):
    pairs = []
    labels = []

    n = min([len(digit_indices[d]) for d in range(10)]) - 1

    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

# 5. Create digit indices
digit_indices_train = [np.where(y_train == i)[0] for i in range(10)]
digit_indices_test = [np.where(y_test == i)[0] for i in range(10)]

# 5. Prepare the pairs
train_pairs, train_labels = create_pairs(x_train, digit_indices_train)

In [6]:
train_labels.shape

(108400,)

In [7]:
# 6. Compile the model
model.compile(loss=contrastive_loss, optimizer='adam')

# 6. Train the model
model.fit([train_pairs[:, 0, :,:], train_pairs[:, 1, :,:]], train_labels, batch_size=128, epochs=10)

Epoch 1/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1408
Epoch 2/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0693
Epoch 3/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0577
Epoch 4/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0524
Epoch 5/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0489
Epoch 6/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0459
Epoch 7/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0442
Epoch 8/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0418
Epoch 9/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0405
Epoch 10/10
847/847 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0393


To evaluate and make predictions with the Siamese Network, we'll add steps to visualize the images being tested along with their predictions. We'll create a function to visualize pairs of images and their similarity score, and we'll also evaluate the model on a test set.

Here's how we can do it:

Prepare the Test Data: Create pairs from the MNIST test set and their labels.

Evaluate the Model: Use the model to evaluate these pairs.

Prediction and Visualization: Make predictions on a subset of these pairs and visualize the results.

In [8]:
# 1. Prepare the Test Data
test_pairs, test_labels = create_pairs(x_test, digit_indices_test)

# 2. Evaluate the model
model.evaluate([test_pairs[:, 0], test_pairs[:, 1]], test_labels)

557/557 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0408


0.04798704385757446

In [9]:
# 3. Make predictions on a sample of test pairs
n = 10  # Number of sample pairs to visualize
sample_pairs = test_pairs[:n, :]
sample_labels = test_labels[:n]
predictions = model.predict([sample_pairs[:, 0], sample_pairs[:, 1]])
print(f'Predictions are {predictions}')
print(f'Labels are {sample_labels}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
Predictions are [[0.03671904]
 [0.81364566]
 [0.03901041]
 [0.78875977]
 [0.04539647]
 [0.7693171 ]
 [0.09018993]
 [1.4512309 ]
 [0.15611188]
 [1.2585396 ]]
Labels are [1 0 1 0 1 0 1 0 1 0]
